In [4]:
import os

In [5]:
pip install langchain langchain-google-genai google-generativeai

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document
import os
import streamlit as st
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import traceback
from dotenv import load_dotenv
import PyPDF2
load_dotenv()

c:\Users\HP\miniconda3\envs\envlc\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


True

In [7]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)  # 👈 important
key = os.getenv("API_KEY")



In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, api_key=key)

### Quiz Chain

In [9]:
Template = """
You are an expert educational content generator.

TASK:
Generate Multiple Choice Questions (MCQs) from the given content.

INPUT DATA:
Text: {text}
Subject: {subject}

INSTRUCTIONS:
- Generate exactly {no_of} MCQs
- Each MCQ must have exactly {number} options
- Only ONE option must be correct
- Difficulty level must be: {tone}
- Questions must NOT repeat
- Questions must NOT be semantically similar
- Questions must be concept-based, not copy-paste lines
- Questions must be clearly framed
- Options must be realistic and confusing (but fair)
- Avoid ambiguous questions
- Avoid multiple correct answers
- Do NOT include explanations unless explicitly asked
- Do NOT include extra text outside JSON

OUTPUT FORMAT:
Strictly follow the JSON structure below.

### RESPONSE_JSON:
{response_json}

IMPORTANT RULES:
- Output must be valid JSON
- No markdown
- No commentary
- No extra keys
- No missing fields
- No trailing commas
- No text outside JSON
"""


In [10]:
with open(r"C:\Users\HP\Documents\Projects\Generative AI\Langchain\response.js", "r") as file:
    response_json = file.read()

In [11]:
from langchain.document_loaders import TextLoader

loader = TextLoader("test.txt", encoding="utf-8")
documents = loader.load()

# Extract text
input_text = documents[0].page_content
print(input_text[0:500])  # Print first 500 characters


What is generative AI?
Generative AI is artificial intelligence designed to create unique text or image results in response to user prompts. The technology uses machine learning to return an output based on the user’s prompt. AI engineers train the technology using large data sets, which the model consults when determining the best possible answer to a prompt. Another way to look at generative AI is as a form of predictive artificial intelligence. Based on the information provided, generative AI


In [12]:
from langchain.prompts import PromptTemplate

quiz_prompt = PromptTemplate(
    input_variables=["text", "subject", "number", "tone", "no_of", "response_json"],
    template=Template
)
quiz_chain = LLMChain(llm=llm, prompt=quiz_prompt)

C:\Users\HP\AppData\Local\Temp\ipykernel_19452\1222439241.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_prompt)


In [13]:
response = quiz_chain.run(
    text=input_text,
    subject="Artificial Intelligence",
    number=4,
    no_of=5,
    tone="medium",
    response_json=response_json)


print(response)


C:\Users\HP\AppData\Local\Temp\ipykernel_19452\1336454056.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = quiz_chain.run(


{
"1":
{
"no": "1",
"question": "What is the fundamental characteristic that defines generative AI?",
"options": {
"A": "Its ability to analyze existing data for statistical trends and patterns.",
"B": "Its design to create unique and novel content, such as text or images, in response to user prompts.",
"C": "Its primary function to manage and organize large databases efficiently for quick retrieval.",
"D": "Its capacity to execute pre-programmed automated tasks without human intervention."
},
"correct_answer": "B"
},
"2":
{
"no": "2",
"question": "Which statement best describes the underlying mechanism by which generative AI produces its output?",
"options": {
"A": "It directly copies and reassembles segments of information from its training datasets.",
"B": "It predicts the most probable sequence of words or elements to form the best answer based on learned patterns.",
"C": "It relies on a fixed set of predefined rules and templates to generate predetermined results.",
"D": "It requi

### Review Chain

In [14]:
# Review Chain
from langchain.prompts import PromptTemplate

review_template = """
You are an expert examiner and subject matter validator.

You will receive:
1) Original source text
2) Generated MCQs in JSON format
3) Difficulty tone

Your task:
Validate each MCQ for:
- Conceptual correctness
- Factual correctness
- One and only one correct answer
- Answer correctness
- No ambiguity
- No hallucination
- Proper difficulty as per tone
- No repeated or similar options
- Logical clarity
- Relevance to the source text

### OUTPUT FORMAT (STRICT JSON ONLY):

{{
  "overall_score": "0-100",
  "summary": "short quality summary",
  "verdict": "PASS / FAIL",
  "errors_count": number,
  "question_reviews": [
    {{
      "question_id": 1,
      "status": "PASS / FAIL",
      "issues": [
        "issue1",
        "issue2"
      ],
      "difficulty_match": true/false,
      "answer_correct": true/false,
      "logic_valid": true/false,
      "hallucination": true/false,
      "improvement_suggestion": "text"
    }}
  ]
}}

Rules:
- Be strict
- Do not rewrite MCQs
- Do not explain outside JSON
- No markdown
- No comments
"""
review_prompt = PromptTemplate(
    input_variables=["source_text", "mcq_json", "tone"],
    template=review_template
)

In [15]:
review_prompt = PromptTemplate(
    input_variables=["text", "mcq_json", "tone"],
    template=review_template
)

from langchain.chains import LLMChain

review_chain = LLMChain(
    llm=llm,   # same or separate LLM
    prompt=review_prompt
)
review_response = review_chain.run(
    text=input_text,
    mcq_json=response,
    tone="medium"
)


import json

review_result = json.loads(review_response)

print(review_result)


{'overall_score': 'N/A', 'summary': 'Awaiting input: Source text, MCQs, and difficulty tone are required to perform the validation.', 'verdict': 'N/A', 'errors_count': 0, 'question_reviews': []}


In [16]:
import json

print(json.dumps(review_result, indent=4))

{
    "overall_score": "N/A",
    "summary": "Awaiting input: Source text, MCQs, and difficulty tone are required to perform the validation.",
    "verdict": "N/A",
    "errors_count": 0,
    "question_reviews": []
}


In [17]:
response = quiz_chain.run({
    "text": "AI is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning (the acquisition of information and rules for using the information), reasoning (using rules to reach approximate or definite conclusions), and self-correction. AI can be categorized into two main types: narrow AI, which is designed to perform a specific task (like voice recognition or playing chess), and general AI, which has the ability to understand, learn, and apply knowledge in a way that is indistinguishable from human intelligence. AI technologies are widely used in various applications such as natural language processing, robotics, computer vision, and expert systems.",
    "subject": "Biology",
    "number": 4,
    "tone": "medium",
    "no_of": 5,
    "response_json": response_json
})

print(response)


{
"1":
{
"no": "1",
"question": "What is the primary goal of Artificial Intelligence (AI) as described in the text?",
"options": {
"A": "To replace human workers with automated systems.",
"B": "To enhance human physical capabilities through robotics.",
"C": "To simulate human intelligence processes using machines.",
"D": "To develop new biological life forms with advanced cognition."
},
"correct_answer": "C"
},
"2":
{
"no": "2",
"question": "According to the text, which of the following is a fundamental process involved in Artificial Intelligence?",
"options": {
"A": "Data encryption",
"B": "Algorithmic optimization",
"C": "Self-correction",
"D": "Network protocol management"
},
"correct_answer": "C"
},
"3":
{
"no": "3",
"question": "A system designed specifically for playing chess or recognizing speech would be classified as which type of AI?",
"options": {
"A": "General AI",
"B": "Advanced AI",
"C": "Narrow AI",
"D": "Cognitive AI"
},
"correct_answer": "C"
},
"4":
{
"no": "4",
"quest

### Fix MCQ

In [18]:
fix_template = """
You are an expert AI question corrector.

You will receive:
1) Original source text
2) Generated MCQs JSON
3) Review report JSON with errors

Your task:
- Fix ONLY the incorrect questions
- Keep correct questions unchanged
- Ensure:
  - Conceptual correctness
  - Factual correctness
  - One correct answer
  - No hallucinations
  - Proper difficulty
  - JSON format consistency
  - Same structure as input MCQ JSON

### OUTPUT FORMAT:
Return ONLY corrected MCQ JSON (no explanation, no markdown).
"""


In [19]:
fix_prompt = PromptTemplate(
    input_variables=["text", "mcq_json", "review_json"],
    template=fix_template
)


In [20]:
fix_chain = LLMChain(
    llm=llm,
    prompt=fix_prompt
)


In [21]:
import json

MAX_RETRIES = 3

def self_correcting_quiz_pipeline(input_text, number, tone, no_of, response_json):
    
    # Step 1: Generate Quiz
    quiz_response = quiz_chain.run({
        "text": input_text,
        "subject": "Artificial Intelligence",
        "number": number,
        "tone": tone,
        "no_of": no_of,
        "response_json": response_json
    })

    current_mcq = quiz_response

    for attempt in range(MAX_RETRIES):

        # Step 2: Review
        review_response = review_chain.run(
            text=input_text,
            mcq_json=current_mcq,
            tone= tone
        )

        review_result = json.loads(review_response)

        print(f"\n🔍 Review Attempt {attempt+1}")
        print("Verdict:", review_result["verdict"])

        # Step 3: Check Verdict
        if review_result["verdict"] == "PASS":
            print("✅ MCQs validated successfully")
            return {
                "status": "SUCCESS",
                "mcq": json.loads(current_mcq),
                "review": review_result
            }

        # Step 4: Fix if FAIL
        print("🛠 Fixing incorrect questions...")

        fixed_mcq = fix_chain.run(
            text=input_text,
            mcq_json=current_mcq,
            review_json=json.dumps(review_result)
        )

        current_mcq = fixed_mcq

    # If still failing after retries
    return {
        "status": "FAILED_AFTER_RETRIES",
        "mcq": json.loads(current_mcq),
        "review": review_result
    }


In [22]:
print("Enter the following details: ")
import ipywidgets as widgets
from IPython.display import display

number_w = widgets.IntText(value=5, description="Questions:")
tone_w = widgets.Dropdown(options=["easy","medium","hard"], description="Tone:")
no_of_w = widgets.IntText(value=4, description="Options:")

display(number_w, tone_w, no_of_w)

# Use values
number = number_w.value
tone = tone_w.value
no_of = no_of_w.value


result = self_correcting_quiz_pipeline(input_text, number, tone, no_of, response_json)

print("\nFINAL STATUS:", result["status"])

print("\nFINAL MCQs:")
print(json.dumps(result["mcq"], indent=4))

print("\nFINAL REVIEW:")
print(json.dumps(result["review"], indent=4))


Enter the following details: 


IntText(value=5, description='Questions:')

Dropdown(description='Tone:', options=('easy', 'medium', 'hard'), value='easy')

IntText(value=4, description='Options:')


🔍 Review Attempt 1
Verdict: PASS
✅ MCQs validated successfully

FINAL STATUS: SUCCESS

FINAL MCQs:
{
    "1": {
        "no": "1",
        "question": "What is the primary function of Generative AI?",
        "options": {
            "A": "To analyze existing data for patterns.",
            "B": "To store vast amounts of information.",
            "C": "To create unique text or image results.",
            "D": "To manage network security.",
            "E": "To execute pre-programmed commands."
        },
        "correct_answer": "C"
    },
    "2": {
        "no": "2",
        "question": "Which of the following is an example of Generative AI mentioned in the text?",
        "options": {
            "A": "Microsoft Word",
            "B": "Google Chrome",
            "C": "DALL-E",
            "D": "Adobe Photoshop",
            "E": "Apple Siri"
        },
        "correct_answer": "C"
    },
    "3": {
        "no": "3",
        "question": "Generative AI is described as a form 

In [ ]:
import json
import pandas as pd

rows = []

for i, q in enumerate(result["mcq"], start=1):
    if isinstance(q, str):
        q = json.loads(q)   # convert string JSON → dict

    rows.append({
        "No": i,
        "Question": q["question"],
        "Options": " | ".join(q["options"]),
        "Answers": q["correct_answer"]
    })

df_mcq = pd.DataFrame(rows)
df_mcq

In [28]:
import json
import pandas as pd

rows = []

for i, q in enumerate(result["mcq"], start=1):
    if isinstance(q, str):
        q = json.loads(q)   # convert string JSON → dict

    rows.append({
        "No": i,
        "Question": q["question"],
        "Options": " | ".join(q["options"]),
        "Answers": q["correct_answer"]
    })

df_mcq = pd.DataFrame(rows)
df_mcq


TypeError: 'int' object is not subscriptable

In [25]:
import pandas as pd
import json

json_str = json.dumps(result["mcq"])
df = pd.read_json(json_str)
print(df.to_csv("mcq_output.csv", index=False))


None


C:\Users\HP\AppData\Local\Temp\ipykernel_19452\2560007267.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_str)
